In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import pyLDAvis
import pyLDAvis.sklearn

import re
from tqdm import tqdm
from tqdm import tnrange
from tqdm import tqdm_notebook
from wordcloud import WordCloud

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF
from sklearn.feature_extraction import stop_words as sklearn_stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE

from gensim import corpora
from gensim import models
from gensim import similarities
from gensim import matutils
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

import pickle
from pymongo import MongoClient
from bson import json_util

/home/cneiderer/anaconda3/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
%matplotlib inline
pyLDAvis.enable_notebook()
# custom_style = {'axes.labelcolor': 'white',
#                 'xtick.color': 'white',
#                 'ytick.color': 'white'}
# sns.set_style("darkgrid", rc=custom_style)

In [44]:
import plotly.plotly as py
import plotly.graph_objs as go
import cufflinks as cf
import pandas as pd
import numpy as np

import plotly
import json
credentials = json.load(open('/home/cneiderer/.plotly/.credentials'))
plotly.tools.set_credentials_file(username=credentials['username'], api_key=credentials['api_key'])

### Presidents List

In [4]:
pres_list = pd.read_csv('https://gist.githubusercontent.com/namuol/2657233/raw/74135b2637e624848c163759be9cd14ae33f5153/presidents.csv')

### JSON-Pandas Stuff

In [5]:
json_file = '/home/cneiderer/Downloads/presidency.json'

In [6]:
# Load presidential json data into df
pres_df = pd.read_json(json_file)
# Convert from wide to long format
pres_df = pres_df.T
# Subset data on oral and written speeches
pres_df = pres_df[['Oral', 'Written']]

In [7]:
# Transform into more usable format
df = pd.DataFrame()
for idx in tnrange(len(pres_df.index)):
    pres = pres_df.index[idx]
    pres_data = pres_df.loc[pres]
    
    speech_type = ['Oral', 'Written']
    for jdx in tnrange(len(speech_type)):
        tmp_df = pd.DataFrame.from_dict(pres_df.loc[pres][speech_type[jdx]])
        tmp_df.insert(0, 'President', pres)
        tmp_df.insert(1, 'Type', speech_type[jdx])
        try:
            tmp_df.document_date = pd.to_datetime(tmp_df.document_date)
        except:
            tmp_df.document_data = pd.to_datetime('20500101')
        df = pd.concat([df, tmp_df], axis=0)
df.index = range(0, df.shape[0])        

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

...

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58595 entries, 0 to 58594
Data columns (total 8 columns):
President        58595 non-null object
Type             58595 non-null object
category         58595 non-null object
content          58595 non-null object
document_date    58595 non-null datetime64[ns]
pid              58595 non-null float64
subcategory      41464 non-null object
title            58595 non-null object
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 3.6+ MB


### Clean Corpus

In [9]:
corpus = df['content']

In [10]:
len(corpus)

58595

In [11]:
# clean and stem presidential addresses
clean_corpus = []
punctuation = re.compile(r"[!#$%&\'\"()\*\+,-\./:;<=>\?@\[\\\]^_`{\|}~]")
# house_senate_opening = re.compile('''to\s+the\s+(?:senate|house\s+of\s+representatives|)
#                                      (\s+and\s+)*(?:senate|house\s+of\s+representatives|house)*''')
for doc in corpus:
    temp = doc.lower()
    temp = re.sub(r'\n', ' ', temp)
    temp = re.sub(r'united\s+states', ' ', temp)
    temp = re.sub(r'mr.\s+(vice)?president', ' ', temp)
    temp = re.sub(r'(?:mr\.|madam)\s+speaker', ' ', temp)
    temp = re.sub(r'americaa', 'america', temp)
    temp = re.sub(r'special\s+message', ' ', temp)
    temp = re.sub(r'executive\s+order', ' ', temp)
    temp = re.sub(r'statement\s+on\s+the', ' ', temp)
    temp = re.sub(r'section\s+\d+', ' ', temp)
#     temp = re.sub(house_senate_opening, temp)
    temp = re.sub(r'[^A-Za-z]+', ' ', temp)
    temp = re.sub(punctuation, ' ', temp)
    
#     temp = word_tokenize(temp)
#     lemma = []
#     for word in temp:
#         lemma.append(stemmer.stem(word))
#     temp = ' '.join(clean_corpus[0])
    
    clean_corpus.append(temp)
    
len(clean_corpus)

58595

In [12]:
# Combine Sklearn and NLTK English stop words
months_list = ['january', 'jan', 'february', 'feb', 'march', 'mar', 'april', 'apr', 'may', 'june', 'jun', 
               'july', 'jul', 'august', 'aug', 'september', 'sept', 'october', 'oct', 'november', 'nov', 'december', 'dec']
custom_stopwords = ['hereunto', 'whereof', 'thereof', 'nineteen', 'said', 'herewith', 'shall', 'say', 'like',
                    'mr', 'mrs', 'sir', 'dear', 'george', 'bush', 'william', 'thirty', 'lot', 'want', 'let', 
                    'sure', 'way', 'section', 'act', 'answer', 'laughter', 'great', 'good', 'things', 'thing',
                    'going', 'know', 'day', 'new', 'make', 'got', 'yes', 'year', 'week', 'sec', 'obama', 'john'] + months_list

stopwords = sklearn_stopwords.ENGLISH_STOP_WORDS \
            .union(nltk_stopwords.words('english')) \
            .union(gensim_stopwords) \
            .union(custom_stopwords)

In [13]:
# Helper Function
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]]), '\n')
    print()

In [14]:
n_features = 1000
n_topics = 20
n_top_words = 20
seed = 129

In [15]:
pickle_file = 'APP_lda_results.pickle'
with open(pickle_file, 'rb') as f:
    data = pickle.load(f)
data.keys()

dict_keys(['tf_vectorizer', 'dtm_tf', 'lda_tf', 'lda_vis', 'lda_tm'])

In [16]:
tf_vectorizer = data['tf_vectorizer']
dtm_tf = data['dtm_tf']
lda_tf = data['lda_tf']
lda_tm = data['lda_tm']
lda_vis = data['lda_vis']

###  Normalize Topic Frequency by Year

In [17]:
lda_tm_df = pd.DataFrame(lda_tm)

In [18]:
lda_tm_df['year'] = df.document_date.dt.year

In [72]:
# Normalize topic frequency by year
topic_freq = lda_tm_df.groupby('year').sum()
topic_freq = 100 * topic_freq.div(topic_freq.sum(axis=1), axis=0)
topic_freq = topic_freq.round(3)

In [73]:
topic_freq.sample(20, random_state=129)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
year,,,,,,,,,,,,,,,,,,,,
1873,0.257,0.972,0.251,5.440,1.935,2.245,0.465,6.922,0.666,3.540,7.521,10.268,0.552,2.446,24.491,0.907,0.896,0.398,27.983,1.844
1960,6.814,8.824,0.985,3.289,3.754,7.157,0.610,6.438,9.117,1.160,7.600,3.566,2.184,5.732,6.618,1.734,6.118,0.993,2.675,14.633
2015,2.563,6.686,2.783,3.051,1.305,1.758,2.682,4.668,8.182,0.494,12.491,2.101,7.451,3.569,2.211,1.360,8.072,16.506,0.868,11.199
1830,1.654,0.346,0.399,1.216,2.148,1.592,0.376,4.896,0.363,6.575,1.236,4.804,0.283,5.771,17.432,1.716,0.283,0.283,47.447,1.180
1968,4.976,5.995,1.943,3.646,1.679,10.548,2.066,3.760,12.585,0.932,5.909,2.945,2.524,5.990,4.891,1.669,8.560,3.006,2.742,13.634
1938,7.497,3.307,1.754,2.476,2.109,13.111,1.023,4.272,7.327,0.867,10.414,3.609,0.929,6.748,12.393,2.460,4.969,2.640,4.724,7.372
1983,3.295,11.353,2.694,3.861,1.530,7.650,1.947,7.145,7.654,0.927,6.447,1.943,5.735,4.002,3.474,1.714,7.276,6.468,4.251,10.633
1924,1.297,2.895,1.503,1.006,0.392,4.794,0.155,12.578,7.177,0.184,6.240,3.391,0.037,3.504,28.730,0.617,3.374,2.276,1.533,18.316
1915,4.778,3.400,3.050,0.408,0.931,1.873,0.196,12.282,3.658,1.008,16.126,5.503,0.169,3.576,18.153,0.664,1.040,1.810,1.873,19.503


### Plot Stacked Area Chart

In [36]:
colors = pd.read_csv('/home/cneiderer/Documents/20distinctcolors.csv', header=None)

In [38]:
colors

,0,1,2,3
0,Red,#e6194b,"(230, 25, 75)","(0, 100, 66, 0)"
1,Green,#3cb44b,"(60, 180, 75)","(75, 0, 100, 0)"
2,Yellow,#ffe119,"(255, 225, 25)","(0, 25, 95, 0)"
3,Blue,#0082c8,"(0, 130, 200)","(100, 35, 0, 0)"
4,Orange,#f58231,"(245, 130, 48)","(0, 60, 92, 0)"
5,Purple,#911eb4,"(145, 30, 180)","(35, 70, 0, 0)"
6,Cyan,#46f0f0,"(70, 240, 240)","(70, 0, 0, 0)"
7,Magenta,#f032e6,"(240, 50, 230)","(0, 100, 0, 0)"
8,Lime,#d2f53c,"(210, 245, 60)","(35, 0, 100, 0)"
9,Pink,#fabebe,"(250, 190, 190)","(0, 30, 15, 0)"


In [37]:
cmap = list(colors[1])

In [79]:
layout = go.Layout(title='Topic Density Over Time',
                   titlefont={'size': 20},
                   yaxis={'ticksuffix': '%'},
                   showlegend=True,
#                    hovermode='closest',
                   autosize=False, height=600, width=1000)
topic_freq.iplot(kind='area', fill=True, hoverinfo='none',
                 filename='LDA_20Topic_Normalized_By_Year',
                 colors=cmap, layout=layout)

In [ ]:
# plt.stackplot(range(topic_df.shape[1]), )